In [ ]:
!pip install -q starrocks "sqlalchemy<3.0"

In [ ]:
# This CATALOG_URL works for the "docker compose" testing and development environment
# Change 'server' if you are not running on "docker compose" (f. ex. 'localhost' if Lakekeeper is running locally).
CATALOG_URL = "http://server:8080/catalog"

STARROCKS_URI = "starrocks://root@starrocks:9030"

# In the provided "docker compose" testing and development environment,
# this warehouse already exists.
# See USER_GUIDE.MD and STORAGE.MD for configuration examples if you
# are running this notebook with your own setup.
# There is also an example in 'Multiple Warehouses.ipynb'.
WAREHOUSE = "demo"
CATALOG = "demo_catalog"
NAMESPACE = "demo_namespace"

# Create Starrocks Catalog

In [ ]:
from sqlalchemy import create_engine

engine = create_engine(STARROCKS_URI)
connection = engine.connect()
connection.execute("DROP CATALOG IF EXISTS demo")

In [ ]:
# Until https://github.com/StarRocks/starrocks/issues/50585#issue-2501162084 is fixed, we have to configure
# the credentials locally if we need a custom s3 endpoint.
connection.execute(
    f"""
    CREATE EXTERNAL CATALOG demo
    PROPERTIES
    (
        "type" = "iceberg",
        "iceberg.catalog.type" = "rest",
        "iceberg.catalog.uri" = "{CATALOG_URL}",
        "iceberg.catalog.warehouse" = "{WAREHOUSE}",
        "aws.s3.region" = "local",
        "aws.s3.enable_path_style_access" = "true",
        "aws.s3.endpoint" = "http://minio:9000",
        "aws.s3.access_key" = "minio-root-user",
        "aws.s3.secret_key" = "minio-root-password"
    )
    """
)
connection.execute("SET CATALOG demo")

## Read and Write Tables

In [ ]:
connection.execute("CREATE DATABASE starrocks_namespace")
connection.execute(
    "CREATE TABLE starrocks_namespace.my_table (my_ints INT, my_floats DOUBLE, strings VARCHAR)"
)
connection.execute(
    "INSERT INTO starrocks_namespace.my_table VALUES (1, 1.0, 'a'), (2, 2.0, 'b')"
)
connection.execute(
    "SELECT * FROM starrocks_namespace.my_table",
).fetchall()
